In [1]:
import numpy as np
import pandas as pd
from pymongo import MongoClient
import mysql.connector
from datetime import datetime
import requests

# Data Analysis Using MongoDB

## Connect to Database

In [2]:
# # MongoDB setup
transaction = MongoClient('mongodb://localhost:27017/')

# Connect to your database
db = transaction['Coinut']

# MySQL setup
mysql_db = mysql.connector.connect(
  user='root', password='liuwinq1', host='localhost', database='coinut')
mysql_cursor = mysql_db.cursor()

## Fetch Data

In [3]:
# Select the collection from the database
collection = db['transaction']

# Fetch all doc data
data1 = collection.find()

In [4]:
# Count the number of documents
num_transactions = collection.count_documents({})
print(f"Number of transactions: {num_transactions}")

Number of transactions: 119544


# Prepare dictionary

### Currency dictionary

In [5]:
# Define the index to currency mapping
currency_index = {
    0: 'BTC', 1: 'USD', 2: 'LTC', 3: 'ETH', 4: 'ETC', 5: 'USDT', 6: 'XMR', 7: 'BCH', 8: 'BTG', 9: 'B2X', 
    10: 'SBTC', 11: 'SGD', 12: 'CAD', 13: 'EUR', 14: 'NTD', 15: 'JPY', 16: 'GBP', 17: 'LCH', 18: 'CHF', 
    19: 'ZEC', 20: 'ADA', 21: 'MYR', 22: 'DAI', 23: 'XSGD', 24: 'XIDR'}

### Currency_pair dictionary

In [6]:
# API endpoint URL
url = 'https://api.coinut.com/spot'

# Send a GET request to the API
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Get the JSON data from the response
    data = response.json()
    
# Initialize an empty dictionary to store the values
pair_dic = {}

# Iterate over the data
for item in data:
    asset = item['asset']
    inst_id = item['inst_id']
    
    # Add the values to the dictionary with asset as the key
    pair_dic[inst_id] = asset

# Print the result
print(pair_dic)

{1: 'LTCBTC', 2: 'ETHBTC', 3: 'ETCBTC', 490569: 'ETHLTC', 490588: 'ETCLTC', 490589: 'BTCUSDT', 490590: 'LTCUSDT', 490591: 'ETHUSDT', 490592: 'ETCUSDT', 852382: 'XMRUSDT', 852383: 'XMRBTC', 852384: 'XMRLTC', 1111976: 'ZECUSDT', 1111977: 'ZECBTC', 1111978: 'ZECLTC', 1801843: 'BTCCAD', 1801844: 'LTCCAD', 1801845: 'ETHCAD', 1885998: 'ZECCAD', 2372505: 'BTCXSGD', 2372506: 'LTCXSGD', 2372507: 'ETHXSGD', 2372508: 'USDTXSGD', 2372510: 'DAIXSGD', 2372511: 'ETCXSGD', 2372512: 'BTCXIDR'}


### Daily_price dictionary

In [7]:
daily_price_dict={}

mysql_cursor.execute("SELECT * FROM price")
rows = mysql_cursor.fetchall()
columns = [column[0] for column in mysql_cursor.description]

for row in rows:
    row_dict = dict(zip(columns, row))
    date = row_dict['time'].strftime('%Y-%m-%d')
    daily_price_dict[date] = {currency: row_dict[currency] for currency in columns if currency != 'time'}

## Transaction Paires Analysis

In [8]:
# Initialize a dictionary to store the counts and amounts
transaction_details = {}
transaction_totals = {}  # Stores total transaction value in SGD for each pair

# Initialize transaction counter
transaction_counter = 0

# Iterate over the transactions
for transaction in data1:
    inst_id = transaction['matchedOrder']['instId']
    pair = pair_dic.get(inst_id, None)
    
    if pair:
        # Split the currency pair
        split_idx = len(pair) // 2 if len(pair) % 2 == 0 else 3
        first_currency, second_currency = pair[:split_idx], pair[split_idx:]

        # Calculate the amounts
        first_currency_amount = float(transaction['fillQty'])
        second_currency_amount = float(transaction['fillPrice']) * float(transaction['fillQty'])

        # If the pair is not yet in the dictionary, add it with the count and amounts
        if pair not in transaction_details:
            transaction_details[pair] = {'Count': 1, first_currency: first_currency_amount, second_currency: second_currency_amount}
        else:
            # If the pair is already in the dictionary, increment the count and add the amounts
            transaction_details[pair]['Count'] += 1
            transaction_details[pair][first_currency] += first_currency_amount
            transaction_details[pair][second_currency] += second_currency_amount
            

        # Convert timestamp to datetime format
        timestamp = transaction['timestamp']
        dt = datetime.utcfromtimestamp(int(timestamp) / 1e6).strftime('%Y-%m-%d %H:%M:%S.%f')
        dt_date = dt.split()[0]  # get the date part
        
        # Initialize fill_qty and base_currency_price
        fill_qty = 0
        base_currency_price = 0

        # Check if we have price data for this date
        if dt_date in daily_price_dict and first_currency in daily_price_dict[dt_date]:
            base_currency_price = daily_price_dict[dt_date][first_currency]
            fill_qty = first_currency_amount  # Using first_currency_amount instead of getting from transaction again

        # Calculate the transaction value in SGD
        transaction_value_sgd = fill_qty * float(base_currency_price)

        # Add the transaction value to the transaction document
        transaction['transaction_value_sgd'] = transaction_value_sgd
        
        # Increment transaction counter
        transaction_counter += 1
        
        # Update the total for the pair
        if pair in transaction_totals:
            transaction_totals[pair] += transaction_value_sgd
        else:
            transaction_totals[pair] = transaction_value_sgd


 # Print the transaction details
for pair, details in transaction_details.items():
    # Get the currency names from the pair
    split_idx = len(pair) // 2 if len(pair) % 2 == 0 else 3
    first_currency, second_currency = pair[:split_idx], pair[split_idx:]
        
    print(f'Pair: {pair}, Count: {details["Count"]}, {first_currency}: {details[first_currency]:.2f}, {second_currency}: {details[second_currency]:.2f}, Total SGD: {transaction_totals[pair]:.2f}')

# Print the transaction counter
print(f'Total transactions processed: {transaction_counter}')

Pair: LTCBTC, Count: 1241, LTC: 739.19, BTC: 2.24, Total SGD: 69789.48
Pair: LTCUSDT, Count: 63237, LTC: 23915.26, USDT: 1569842.06, Total SGD: 1578911.09
Pair: ETHXSGD, Count: 1323, ETH: 331.84, XSGD: 859274.85, Total SGD: 628359.16
Pair: BTCXSGD, Count: 2778, BTC: 36.08, XSGD: 1340458.01, Total SGD: 1040119.10
Pair: ZECLTC, Count: 1170, ZEC: 6.50, LTC: 7.89, Total SGD: 746.43
Pair: ETHLTC, Count: 782, ETH: 2.11, LTC: 48.66, Total SGD: 4995.18
Pair: ETCLTC, Count: 11890, ETC: 283.93, LTC: 82.32, Total SGD: 6205.93
Pair: XMRUSDT, Count: 3375, XMR: 216.16, USDT: 41943.87, Total SGD: 41835.84
Pair: USDTXSGD, Count: 4522, USDT: 3718059.37, XSGD: 5056328.39, Total SGD: 3718059.37
Pair: LTCXSGD, Count: 779, LTC: 1787.02, XSGD: 203744.84, Total SGD: 149514.09
Pair: ETHUSDT, Count: 5122, ETH: 318.73, USDT: 865657.62, Total SGD: 865165.51
Pair: ZECUSDT, Count: 2111, ZEC: 283.12, USDT: 34812.82, Total SGD: 35002.04
Pair: ETCUSDT, Count: 3340, ETC: 3211.60, USDT: 81117.41, Total SGD: 80903.83
Pa